# [Day 5: Hydrothermal Venture](https://adventofcode.com/2021/day/5)

In [1]:
import collections as cl
import dataclasses as dc
import re

## Part 1

In [2]:
example_data = [
    "0,9 -> 5,9",
    "8,0 -> 0,8",
    "9,4 -> 3,4",
    "2,2 -> 2,1",
    "7,0 -> 7,4",
    "6,4 -> 2,0",
    "0,9 -> 2,9",
    "3,4 -> 1,4",
    "0,0 -> 8,8",
    "5,5 -> 8,2",
]

```
  0123456789
0 .......1..
1 ..1....1..
2 ..1....1..
3 .......1..
4 .112111211
5 ..........
6 ..........
7 ..........
8 ..........
9 222111....
```

`x1,y1 -> x2,y2` = `"0,9 -> 5,9"` = column 0, row 9 -> column 5, row 9
  * x = columns
  * y = rows

In [3]:
@dc.dataclass
class Line:
    x1: int
    y1: int
    x2: int
    y2: int

    @staticmethod
    def parse_line(line):
        if mt_line := re.match(r"^\s*(?P<x1>\d+)\s*\,\s*(?P<y1>\d+)\s*\-\>\s*(?P<x2>\d+)\s*\,\s*(?P<y2>\d+)\s*$", line):
            return Line(int(mt_line["x1"]), int(mt_line["y1"]), int(mt_line["x2"]), int(mt_line["y2"]))

print(f"Check parse_line (valid): {Line.parse_line(example_data[0]) == Line(0, 9, 5, 9)}")
print(f"Check parse_line (invalid): {Line.parse_line('no line') is None}")

Check parse_line (valid): True
Check parse_line (invalid): True


In [4]:
@dc.dataclass
class Diagram1:
    data: list[list[int]] = dc.field(default_factory=list)  # x=rows, y=columns

    def __post_init__(self):
        self.data = [[0]]

    def _resize(self, max_x, max_y):
        min_x_size, min_y_size = max_x + 1, max_y + 1
        if len(self.data[0]) < min_x_size:
            n_extend = min_x_size - len(self.data[0])
            for row in self.data:
                row.extend([0] * n_extend)
        if len(self.data) < min_y_size:
            row_length = len(self.data[0])
            for _ in range(len(self.data), min_y_size):
                self.data.append([0] * row_length)

    def _plot_line(self, line):
        x_delta = line.x2 - line.x1
        y_delta = line.y2 - line.y1
        if (x_delta != 0) and (y_delta == 0):
            for x in range(min(line.x1, line.x2), max(line.x1, line.x2)+1):
                self.data[line.y1][x] += 1
        elif (y_delta != 0) and (x_delta == 0):
            for y in range(min(line.y1, line.y2), max(line.y1, line.y2)+1):
                self.data[y][line.x1] += 1
                
    def add_line(self, line_text):
        line = Line.parse_line(line_text)
        if line is not None:
            self._resize(line.x1, line.y1)
            self._resize(line.x2, line.y2)
            self._plot_line(line)

    def process_lines(self, line_data):
        for line in line_data:
            self.add_line(line)

    def get_at_least_n_overlap(self, n):
        cnt = cl.Counter([col for row in self.data for col in row])
        return sum([count for key, count in cnt.items() if key >= n])

In [5]:
diagram = Diagram1()
diagram.process_lines(example_data)
print(f"Check part 1: {diagram.get_at_least_n_overlap(2) == 5}")

Check part 1: True


In [6]:
with open(r"..\data\Day 05 input.txt", "r") as fh_in:
    line_data = fh_in.readlines()
print(f"Input line check: {len(line_data) == 500}")

Input line check: True


In [7]:
diagram = Diagram1()
diagram.process_lines(line_data)
print(f"Answer part 1: {diagram.get_at_least_n_overlap(2)}")

Answer part 1: 4993


## Part 2

In [8]:
@dc.dataclass
class Diagram2(Diagram1):
    def _plot_line(self, line):
        x, y = line.x1, line.y1
        x_incr = 1 if x < line.x2 else -1
        y_incr = 1 if y < line.y2 else -1
        while True:
            self.data[y][x] += 1
            if (x == line.x2) and (y == line.y2):
                break
            if x != line.x2:
                x += x_incr
            if y != line.y2:
                y += y_incr

In [9]:
diagram = Diagram2()
diagram.process_lines(example_data)
print(f"Check part 2: {diagram.get_at_least_n_overlap(2) == 12}")

Check part 2: True


In [10]:
diagram = Diagram2()
diagram.process_lines(line_data)
print(f"Answer part 2: {diagram.get_at_least_n_overlap(2)}")

Answer part 2: 21101
